# BTTH07: Ẩn dữ liệu khả nghịch trên ảnh bằng phương pháp DE (phần 1)

TODO: Võ Phương Hòa - 1412192

---

## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Mục tiêu chính ở đây là học, học một cách chân thật; nếu bạn được 5 điểm nhưng bạn làm bài một cách chân thật thì vẫn tốt hơn nhiều so với 10 điểm mà không chân thật. Bạn có thể thảo luận với các bạn khác; nhưng bài làm phải là của chính bạn, dựa trên sự hiểu của chính bạn, lời văn là của chính bạn, code là của chính bạn. Bạn không nên đọc bài làm của các bạn năm trước. Nếu bị phát hiện gian lận thì bạn sẽ bị 0 điểm cho toàn bộ phần thực hành.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt: file `Ex07-ReversibleDataHiding-Img-DE1.ipynb`, và các file dữ liệu mà mình đính kèm (`expandable_cover.bmp`, `msg.txt`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import bitarray
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
# You can also import other things ...

## 3. Hàm nhúng

Trong tuần này, bạn sẽ cài đặt hàm nhúng và hàm rút trích với ảnh đầu vào gồm các cặp pixel đều có h "expandable". Lưu ý là, với file `expandable_cover.bmp` mà mình cung cấp, để đảm bảo tất cả các h đều "expandable" thì bạn phải tiến hành gom 2 pixel thành một nhóm theo cách thức sau: lấy các cột của ma trận pixel nối lại với nhau thành một vector duy nhất, rồi gom 2 pixel kề nhau thành một nhóm.

In [2]:
def embed_DE(cover_img_file, msg_file, stego_img_file):
    """
    Embeds a message into a cover image using the DE (Difference Expansion) method.
    
    Parameters
    ----------
    cover_img_file : string
        The name of the cover img file.
    msg_file : string
        The name of the message file.
    stego_img_file : string
        The name of the stego img file.
    """
    # TODO
    # 1. Read cover img file
    cover_img = imread(cover_img_file)
    
    # 2. Read msg file and process
    # 2.1. Read msg file
    f = open(msg_file, 'r')
    msg_text = f.read()
    f.close()
    
    # 2.2 Convert msg to msg bits
    msg_bits = bitarray.bitarray()
    msg_bits.frombytes(msg_text)
    
    # 2.3. Check if it is possible to embed
    len_msg_bits = len(msg_bits)
    max_num_bits = cover_img.size / 2
    if (len_msg_bits >= max_num_bits):
        return False
    
    # 2.4. Add '100...' to msg bits
    msg_bits.extend('1' + '0' * (max_num_bits - len_msg_bits - 1))
    
    # 3. Embed msg bits into cover img
    stego_img = cover_img.T.reshape(cover_img.size)
    idx_msg = 0
    for i in range(0, len(stego_img), 2):
        h = abs(int(stego_img[i]) - int(stego_img[i + 1])) * 2 + msg_bits[idx_msg]
        l = (int(stego_img[i]) + stego_img[i + 1]) / 2
        x = l + (h + 1) / 2
        y = l - h / 2
        stego_img[i] = x * (stego_img[i] >= stego_img[i + 1]) + y * (stego_img[i] < stego_img[i + 1])
        stego_img[i + 1] = x + y - stego_img[i]
        idx_msg += 1
    stego_img = stego_img.reshape(len(cover_img), len(cover_img[0])).T
    
    # 4. Write stego img to file
    imsave(stego_img_file, np.asarray(stego_img).astype('uint8'))
    
    return True

## 4. Hàm rút trích

In [3]:
def extract_DE(stego_img_file, extracted_msg_file, recovered_cover_img):
    """
    Extracts the message from a stego image using the DE method.
    
    Parameters
    ----------
    stego_img_file : string
        The name of the stego img file.
    extracted_msg_file : string
        The name of the extracted message file.
    recovered_cover_img_file : string
        The name of the recovered cover img file.
    """
    # TODO
    # 1. Read stego img file
    stego_img = imread(stego_img_file)
    
    # 2.Extracted msg str (string of bits of msg file) and recovered cover img from stego img
    cover_img = stego_img.T.reshape(stego_img.size)
    extracted_msg_str = ''
    for i in range(0, len(cover_img), 2):
        h = abs(int(cover_img[i]) - int(cover_img[i + 1]))
        l = min(cover_img[i], cover_img[i + 1]) + h / 2
        extracted_msg_str += str(h % 2)
        h = h / 2
        x = l + (h + 1) / 2
        y = l - h / 2
        cover_img[i] = x * (cover_img[i] >= cover_img[i + 1]) + y * (cover_img[i] < cover_img[i + 1])
        cover_img[i + 1] = x + y - cover_img[i]
        
    cover_img = cover_img.reshape(len(stego_img), len(stego_img[0])).T
    
    # 3. Format extracted msg text
    # 3.1. Find index of the last bit 1 of extracted msg str,
    # create a new string with content similar to the old string
    # but eliminate the last bits '1000 ...' based on the index of the last bit 1
    index = len(extracted_msg_str) - 1     
    while index >= 0:
        if extracted_msg_str[index] == '1':
            break
        index -= 1
    extracted_msg_str = extracted_msg_str[0:index]
    
    # 3.2. Convert extracted msg bit
    extracted_msg_bits = bitarray.bitarray(extracted_msg_str)
    extracted_msg_text = extracted_msg_bits.tobytes()
    
    # 4. Write the results to file
    # 4.1. Write extracted msg text to file
    f = open(extracted_msg_file, 'w')
    f.write(extracted_msg_text)
    f.close()
    
    # 4.2. Write recovered cover img to file
    imsave(recovered_cover_img, np.asarray(cover_img).astype('uint8'))
       
    return

## 5. Chạy thí nghiệm

Bạn sẽ gọi hàm nhúng và hàm rút trích ở trên với file cover `expandable_cover.bmp` và file message `msg.txt` mà mình đính kèm. Để minh chứng hàm nhúng và hàm rút trích đã được cài đặt đúng, bạn cần mở file message ban đầu và file message được rút trích, và in nội dung của hai file này ra; ngoài ra, bạn cũng cần đọc ảnh cover ban đầu và ảnh cover khôi phục được, và so sánh hai mảng ứng với hai ảnh này với nhau (dùng hàm `np.array_equal`).

In [4]:
# TODO
input_1 = 'expandable_cover.bmp'
input_2 = 'msg.txt'
output = 'stego.bmp'
output_1 = 'extracted.txt'
output_2 = 'recovered.bmp'

b = embed_DE(input_1, input_2, output)
if b == True:
    extract_DE(output, output_1, output_2)
    
    f = open(input_2, 'r')
    msg_text = f.read()
    f.close()
    
    f = open(output_1, 'r')
    extracted_text = f.read()
    f.close()
    
    a = imread(input_1)
    b = imread(output_2)
    
    print msg_text
    print '------------------------------'
    print extracted_text
    print '------------------------------'
    print np.array_equal(a, b)

Empty your mind of all thoughts.
Let your heart be at peace.
------------------------------
Empty your mind of all thoughts.
Let your heart be at peace.
------------------------------
True


## 6. Các câu hỏi lý thuyết

Trước hết, bạn đọc các mục sau trong bài báo đính kèm: mục II, mục A và B trong mục III. Phần đọc này ngắn, dễ đọc, và mình đã giảng ở trên lớp; bạn cần nắm chắc để tạo nền tảng cho tuần sau.

Sau đó, bạn trả lời các câu hỏi sau:

- Công thức DE?

TODO

Cho $x, y \in [0, 255]$

$l = \left \lfloor \frac{x + y}{2} \right \rfloor$

$h = x - y$

$\color{red}{h^{'} = 2h + b}$ (với b là bit cần nhúng) (công thức DE)

$x^{'} = l + \left \lfloor \frac{h^{'} + 1}{2} \right \rfloor$

$y^{'} = l - \left \lfloor \frac{h^{'}}{2} \right \rfloor$

- Để x, y nằm trong đoạn [0, 255] thì h phải thỏa điều kiện gì?

TODO

Với $l = \left \lfloor \frac{x + y}{2} \right \rfloor, h = x - y$, ta suy ra $x = l + \left \lfloor \frac{h + 1}{2} \right \rfloor, y = l - \left \lfloor \frac{h}{2} \right \rfloor$


 $0 \le x, y \le 255 \Leftrightarrow \begin{align} \begin{cases} 0 \le l + \left \lfloor \frac{h + 1}{2} \right \rfloor \le 255 \\ 0 \le l - \left \lfloor \frac{h}{2} \right \rfloor \le 255 \end{cases} \end{align} \Leftrightarrow$ $\begin{align} \begin{cases} |h| \le 2(255 - l), &if 128 \le l \le 255. \\ |h| \le 2l + 1, &if 0 \le l \le 127. \end{cases} \end{align}$
 
Tổng quát: $\color{red}{|h| \le min(2(255 - l), 2l + 1)}$ **(1)**

- h thế nào thì được gọi là "expandable"?

TODO

h được gọi là "expandable" $\Leftrightarrow h^{'} = 2h + b$ thoả **(1)** với mọi b (b là bit cần nhúng, b = 0 hoặc b = 1)

- h thế nào thì được gọi là "changeable"?

TODO

h được gọi là "changeable" $\Leftrightarrow h^{'} = 2\left\lfloor\frac{h}{2}\right\rfloor + b$ thoả **(1)** với mọi b (b là bit cần nhúng, b = 0 hoặc b = 1)

- Nếu h "changeable" thì h' = 2floor(h/2) + b có "changeable" không (floor là phép làm tròn xuống)? Giải thích.

TODO

Có.

Giải thích: 

h' changeable nếu $\color{red}{h^{''} = 2\left\lfloor\frac{h'}{2}\right\rfloor + b}$ **(*)** thoả **(1)** với mọi b

Thế $h^{'} = 2\left\lfloor\frac{h}{2}\right\rfloor + b$ vào **(*)**, ta được:

**(*)** $\Leftrightarrow h^{''} = 2\left\lfloor\frac{2\left\lfloor\frac{h}{2}\right\rfloor + b}{2}\right\rfloor + b = 2\left\lfloor\left\lfloor\frac{h}{2} \right\rfloor + \frac{b}{2}\right\rfloor + b = 2\left\lfloor\frac{h}{2}\right\rfloor + b $ thỏa **(1)** với mọi b (do h "changeable")

Do đó, h' "changeable".

- Nếu h "expandable" thì h có "changeable" không? Giải thích.

TODO

Có.

Giải thích:

Do h "expandable" nên $h^{'} = 2h + b$ thỏa **(1)**, tức là $|h'| = |2h + b| \le min(2(255 - l), 2l + 1) \color{red}{(a)}$

Để h "changeable", ta cần chứng minh $2\left\lfloor\frac{h}{2}\right\rfloor + b$ cũng thỏa **(1)**

Dễ thấy: $h \ge \left \lfloor \frac{h}{2} \right \rfloor$ với mọi h > 0 

$\Rightarrow 2h + b \ge 2\left\lfloor\frac{h}{2}\right\rfloor + b$ với mọi h > 0

$\Rightarrow \left|2h + b\right| \ge \left| 2\left\lfloor\frac{h}{2}\right\rfloor + b \right|$ với mọi h > 0 $\color{red}{(b)}$

Từ $\color{red}{(a)}$ và $\color{red}{(b)}$, ta suy ra $\left| 2\left\lfloor\frac{h}{2}\right\rfloor + b \right| \le min(2(255 - l), 2l + 1)$.

Do đó, nếu h "expandable" thì h cũng "changeable"

- Nếu h "expandable" thì h' = 2h + b có "changeable" không? Giải thích.

TODO

Có.

Giải thích:

$h' = 2h + b$ "changeable" nếu $h^{''} = 2\left\lfloor\frac{h'}{2}\right\rfloor + b$ **(*)** thỏa **(1)** với mọi b

Thế $h^{'} = 2h + b$ vào **(*)**, ta được:

**(*)** $\Leftrightarrow h^{''} = 2\left\lfloor\frac{2h + b}{2}\right\rfloor + b = 2\left\lfloor\ h + \frac{b}{2}\right\rfloor + b = 2h + b $ thỏa **(1)** với mọi b (do h "expandable")

Do đó, h' "changeable".


- Chứng minh: nếu h = 0 hoặc h = -1 thì hai khái niệm "expandable" và "changeable" là tương đương nhau.

TODO

Nếu h "expandable" thì $h' = 2h + b$ thỏa **(1)**
    
Nếu h "changeable" thì $h' = 2\left\lfloor\frac{h}{2}\right\rfloor + b$ thỏa **(1)**

Với $h = 0$, cả hai trường hợp trên đều cho kết quả $h' = b$.

Với $h = -1$, cả hai trường hợp trên đều cho kết quả $h' = b - 2$.

Do đó, với h = 0 hoặc h = -1, các khái niệm "expandable" và "changeable" là tương đương nhau.

---

Enjoy DE :-)